# Using Stochastic Gradient Descent On Linear Regression To Predict The Best Time To Go To The Gym

## 1. Introduction

We will use the "Crowdedness at the Campus Gym" dataset and the stochastic gradient descent linear regression (SGDRegressor) algorithm from the scikit-learn library to build and train a model for predicting the best time to go to the gym and avoid crowds.

The dataset contains 11 columns with the following information.

* `number_people`: this is the number of people at the gym at each observation. This will be our target variable or label.
* `date`: a string value with the specific date and time information.
* `timestamp`: an integer (int), with the number of seconds since the start of the day (00:00).
* `day_of_week`: an integer (int). 0 is equal to Monday and 6 is equal to Sunday.
* `is_weekend`: a Boolean value defining if this observation happened during a weekend. 1 for yes, 0 for no.
* `is_holiday`: a Boolean value defining if the observation happened during a holiday. 1 for yes, 0 for no.
* `temperature`: a float, defining the temperature during the day of the observation in Fahrenheit.
* `is_start_of_semester`: a Boolean defining if the observation happened in the first 2 weeks of a semester. 1 for yes, 0 for no.